In [7]:
#!pip install nltk

In [17]:
import os
import requests
from bs4 import BeautifulSoup
import ast
import re
import numpy as np

In [10]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import nltk
nltk.download('punkt')
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\astanchi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [4]:
websites = []
root_path = os.getcwd()
file_input = os.path.join(root_path, 'yelp_closed_biz.txt')
with open(file_input,'r') as fh:
    websites = fh.read().splitlines()
websites

['https://www.yelp.com/biz/maxs-restaurants-san-francisco-3',
 'https://www.yelp.com/biz/franceschis-san-francisco',
 'https://www.yelp.com/biz/butterfly-san-francisco',
 'https://www.yelp.com/biz/la-terrasse-san-francisco',
 'https://www.yelp.com/biz/titas-restaurant-san-francisco',
 'https://www.yelp.com/biz/the-public-san-francisco',
 'https://www.yelp.com/biz/tin-pan-san-francisco',
 'https://www.yelp.com/biz/eat-restaurant-san-francisco',
 'https://www.yelp.com/biz/bonanza-restaurant-san-francisco',
 'https://www.yelp.com/biz/the-metro-bar-and-restaurant-san-francisco',
 'https://www.yelp.com/biz/sinbads-pier2-restaurant-san-francisco-2',
 'https://www.yelp.com/biz/the-monte-carlo-san-francisco',
 'https://www.yelp.com/biz/myth-san-francisco',
 'https://www.yelp.com/biz/2223-restaurant-san-francisco',
 'https://www.yelp.com/biz/the-palace-san-francisco',
 'https://www.yelp.com/biz/watergate-san-francisco',
 'https://www.yelp.com/biz/tallula-san-francisco',
 'https://www.yelp.com/b

In [20]:
def cleanText(dirtyText):
    text = dirtyText.replace('null','""')
    text = text.replace('\t','')
    text = re.sub(' +',' ',text) #replace multiple white spaces with one
    return text[1:]

In [25]:
def analyzeYelp(yelp_dict):
    print ("\n\n************************************************************")
    print("Restaurant: %s" % yelp_dict["name"])
    print("Yelp Rating: %s" % yelp_dict["aggregateRating"]["ratingValue"])
    restaurantSentiments = []
    for idx, review in enumerate(yelp_dict["review"]):
        print("\nReview #%s: %s\n" % (idx+1,review["description"]))
        sentence_list = tokenize.sent_tokenize(review["description"])
        paragraphSentiments=0.0
        for sentence in sentence_list:
            vs = analyzer.polarity_scores(sentence)
            print("{:-<100} {}".format(sentence, str(vs["compound"])))
            paragraphSentiments += vs["compound"]
        avgpSent = round(paragraphSentiments/len(sentence_list),4)
        print("AVERAGE SENTIMENT FOR PARAGRAPH: \t" + str(avgpSent))
        restaurantSentiments.append(avgpSent)

    print("\n\nAVERAGE YELP REVIEW SENTIMENT: \t" + str(round(np.mean(restaurantSentiments),4)))
    print ("************************************************************\n\n")

In [27]:
for url in websites[0:3]:
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
    stuff = soup.find("script", type="application/ld+json")
    good_stuff = cleanText(stuff.text)
    stuff1_dict = ast.literal_eval(good_stuff)
    analyzeYelp(stuff1_dict)



************************************************************
Restaurant: Max’s Restaurants
Yelp Rating: 3.0

Review #1: Why am I writing a review for a closed restaurant?

Well, back in the day they made good matzoh ball soup. For $3.50 I could get a cup with 2 balls and some pickle spears and a roll and call it a meal. Then they cut back on the balls, and I only could get 1 ball. And then I stopped going. Shortly thereafter Max's closed. COINCIDENCE?...

Why am I writing a review for a closed restaurant?-------------------------------------------------- 0.0
Well, back in the day they made good matzoh ball soup.---------------------------------------------- 0.6124
For $3.50 I could get a cup with 2 balls and some pickle spears and a roll and call it a meal.------ 0.0
Then they cut back on the balls, and I only could get 1 ball.--------------------------------------- -0.2732
And then I stopped going.--------------------------------------------------------------------------- -0.2263
Sh